# BDSky Serial's Output Figures & Stats

**A note on Birth Death Skyline Models**
The results in this notebook are from a phylodynamics pipeline using Birth Death Skyline Models. Reading material on Birth Death Skyline Models can be found at:
* [Taming the BEAST Tutorial: Skyline-plots](https://taming-the-beast.org/tutorials/Skyline-plots/)
* [Stadler et al. 2012 PNAS](https://www.pnas.org/doi/full/10.1073/pnas.1207965110)


In [ ]:
save_dir = None
beast_xml_path = None

In [ ]:
import warnings
warnings.filterwarnings( "ignore", module = "matplotlib\..*" )
from beast_pype.outputs import (read_log_file, plot_skyline, plot_hist_kde, plot_origin_or_tmrca, hdi_columns_starting_with, outputs_for_possible_skyline)
from beast2xml import BEAST2XML
from beast_pype.date_utilities import date_to_decimal
import os
import pandas as pd

In [ ]:
if save_dir is None:
    save_dir = os.getcwd()

In [ ]:
beast_xml = BEAST2XML(beast_xml_path)
youngest_tip_year_decimal = beast_xml.extract_youngest_year_decimal()
try:
    sampling_prop_partitions = youngest_tip_year_decimal - beast_xml.extract_rate_change_reverse_times('samplingRateChangeTimes')
except:
    sampling_prop_partitions = None
try:
    rt_partitions = youngest_tip_year_decimal - beast_xml.extract_rate_change_reverse_times('birthRateChangeTimes')
except:
    rt_partitions = None
#Load Trace:
log_file = f'{save_dir}/merged.log'
csv_file = f'{save_dir}/merged_log.csv'
if os.path.isfile(log_file):
    trace_file = log_file
else:
    trace_file = csv_file
trace_df = read_log_file(trace_file, convert_become_uninfectious_rate=True, youngest_tip_date=youngest_tip_year_decimal)

## Infection Period 

BD Skyline models estimate the rate of becoming uninfectious (whose inverse if the average infection period). 

In [ ]:
infectious_fig, ax, infectious_hdi = plot_hist_kde(trace_df=trace_df, parameter='Infection period (per day)', hdi_prob=0.95)
display(infectious_hdi)

# Sampling Proportion

In [ ]:
sampling_prop_cols = [column for column in trace_df.columns if column.startswith('samplingProportion')]
if len(sampling_prop_cols) > 1:
    sampling_fig, sampling_ax, sampling_hdi = plot_skyline(trace_df,
                                                           youngest_tip_year_decimal,
                                                           parameter_start='samplingProportion',
                                                           y_label='Sampling Proportion ($\psi$)',
                                                           grid_size=100,
                                                           include_grid=True,
                                                           partition_year_decimals=sampling_prop_partitions)
else:
    sampling_fig, sampling_ax, sampling_hdi = outputs_for_possible_skyline(trace_df, youngest_tip_year_decimal, sampling_prop_cols[0], axis_label='Sampling Proportion ($\psi$)')
display(sampling_hdi)

# Origin

The origin is the time at which the index case (the first case) became infected, which is slightly earlier than the time-to-the-most-recent-common-ancestor (TMRCA). 

In [ ]:
fig, ax, hdi_est =  plot_origin_or_tmrca(trace_df, parameter='Origin', hdi_prob=0.95)
display({key: value.strftime('%Y-%m-%d') for key, value in hdi_est.items()})

# TMRCA

In [ ]:
fig, ax, hdi_est =  plot_origin_or_tmrca(trace_df, parameter='TMRCA', hdi_prob=0.95)
display({key: value.strftime('%Y-%m-%d') for key, value in hdi_est.items()})

# $R_T$

In [ ]:
rt_fig, rt_ax, rt_hdi = plot_skyline(trace_df,
                                     youngest_tip_year_decimal,
                                     parameter_start='reproductiveNumber',
                                     y_label='$R_t$',
                                     grid_size=100,
                                     include_grid=True,
                                     partition_year_decimals=rt_partitions)
display(rt_hdi)